In [25]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset
from tqdm import tqdm


# ======================
# 1. 数据预处理
# ======================
class TimeSeriesDataset(Dataset):
    def __init__(self, data, look_back=60, forecast_horizon=1):
        self.look_back = look_back
        self.forecast_horizon = forecast_horizon
        self.scaler = MinMaxScaler(feature_range=(0, 1))  #将数据线性缩放到 0 到 1 的区间内

        # 数据归一化
        scaled_data = self.scaler.fit_transform(data)

        # 创建滑动窗口数据集
        X, y = [], []
        for i in range(len(scaled_data) - look_back - forecast_horizon + 1):
            X.append(scaled_data[i:i + look_back, :])
            y.append(scaled_data[i + look_back + forecast_horizon - 1, 3])  # 预测Close价格

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# 读取数据
df = pd.read_csv('./data/USDCNHSP.csv', parse_dates=['date'], index_col='date')

# 特征工程
df['ohlc_range'] = df['high'] - df['low']
df['close_change'] = df['close'] - df['close'].shift(1)
print(df.head())
features = df[['open', 'high', 'low', 'close', 'ohlc_range', 'close_change']].values

# 划分数据集
train_size = int(len(features) * 0.8)
train_data = features[:train_size]
test_data = features[train_size:]

# 创建数据集对象
look_back = 60
dataset = TimeSeriesDataset(train_data, look_back=look_back)
test_dataset = TimeSeriesDataset(test_data, look_back=look_back)

# 数据加载器
batch_size = 256
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

                       symbol     open     high      low    close  ohlc_range  \
date                                                                            
2012-12-31 18:00:00  USDCNHSP  6.22118  6.22153  6.22113  6.22117     0.00040   
2012-12-31 19:00:00  USDCNHSP  6.22112  6.22141  6.22093  6.22137     0.00048   
2012-12-31 20:00:00  USDCNHSP  6.22098  6.22137  6.22043  6.22137     0.00094   
2012-12-31 21:00:00  USDCNHSP  6.22062  6.22142  6.22062  6.22141     0.00080   
2012-12-31 22:00:00  USDCNHSP  6.22150  6.22182  6.22071  6.22126     0.00111   

                     close_change  
date                               
2012-12-31 18:00:00           NaN  
2012-12-31 19:00:00       0.00020  
2012-12-31 20:00:00       0.00000  
2012-12-31 21:00:00       0.00004  
2012-12-31 22:00:00      -0.00015  


In [28]:

# ======================
# 2. TCN模型定义
# ======================
class TemporalConvNet(nn.Module):
    def __init__(self, input_channels, num_layers=4, kernel_size=3, dilation_factors=[1, 2, 4, 8]):
        """

        :param input_channels:
        :param num_layers:
        :param kernel_size:
        :param dilation_factors: 膨胀因子
        """
        super(TemporalConvNet, self).__init__()
        layers = []
        num_channels = input_channels

        for i in range(num_layers):
            dilation = dilation_factors[i] if i < len(dilation_factors) else 1
            padding = (kernel_size - 1) * dilation  #保持时序因果性

            # 因果卷积层  + ReLU
            conv = nn.Conv1d(num_channels, num_channels, kernel_size, padding=padding, dilation=dilation)
            layers += [conv, nn.ReLU()]

            # 残差连接
            if num_channels != input_channels:
                layers += [nn.Conv1d(input_channels, num_channels, 1)]

        self.network = nn.Sequential(*layers)
        self.downsample = nn.Conv1d(input_channels, num_channels, 1) if num_channels != input_channels else None

    def forward(self, x):
        out = self.network(x)
        if self.downsample is not None:
            x = self.downsample(x)
        return nn.ReLU()(out + x)


class TCN(nn.Module):
    def __init__(self, input_dim=6, output_dim=1, num_channels=[64, 64, 64, 64]):
        super(TCN, self).__init__()
        self.encoder = TemporalConvNet(input_dim, num_layers=4, kernel_size=3)
        self.fc = nn.Linear(num_channels[-1], output_dim)

    def forward(self, x):
        # 输入形状: (batch, seq_len, features) → (batch, features, seq_len)
        x = x.transpose(1, 2)
        x = self.encoder(x)
        x = x.mean(dim=2)  # 全局平均池化
        return self.fc(x)


print("模型定义完毕")

模型定义完毕


In [29]:
import matplotlib.pyplot as plt

# 初始化模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TCN(input_dim=6).to(device)

# ======================
# 3. 训练流程
# ======================
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  #学习率为
best_loss = float('inf')

# 早停机制
early_stop = {
    'patience': 10,
    'counter': 0,
    'best_loss': float('inf'),
    'wait': 0
}

# 训练循环
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    loop = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}')
    for inputs, targets in loop:
        # print(inputs,targets)
        inputs, targets = inputs.to(device), targets.to(device)

        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 统计损失
        epoch_loss += loss.item() * inputs.size(0)
        loop.set_postfix(loss=loss.item())

    epoch_loss /= len(train_loader.dataset)

    # 验证
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, targets).item() * inputs.size(0)

    val_loss /= len(test_loader.dataset)

    # 早停判断
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        early_stop['wait'] = 0
    else:
        early_stop['wait'] += 1
        if early_stop['wait'] >= early_stop['patience']:
            print(f'Early stopping at epoch {epoch + 1}')
            break

print(f'Best Validation Loss: {best_loss:.4f}')

# ======================
# 4. 预测与可视化
# ======================
model.load_state_dict(torch.load('best_model.pth'))
model.eval()


# 预测函数
def predict(model, data_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, _ in data_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            predictions.extend(outputs.cpu().numpy())
    return np.array(predictions)


# 反归一化
test_loader_full = DataLoader(test_dataset, batch_size=1024, shuffle=False)
y_true = []
y_pred = []

for inputs, targets in tqdm(test_loader_full):
    y_true.extend(targets.cpu().numpy())
    y_pred.extend(predict(model, DataLoader([inputs], batch_size=1))[0])

scaler = MinMaxScaler(feature_range=(0, 1))
# 反归一化
y_true = scaler.inverse_transform(np.array(y_true)[:, 3].reshape(-1, 1))
y_pred = scaler.inverse_transform(np.array(y_pred)[:, 3].reshape(-1, 1))

# 可视化
plt.figure(figsize=(16, 8))
plt.plot(y_true, label='Actual Price')
plt.plot(y_pred, label='TCN Prediction')
plt.title('Stock Close Price Prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()

Epoch 1/100:   0%|          | 0/226 [00:05<?, ?it/s]


RuntimeError: The size of tensor a (90) must match the size of tensor b (60) at non-singleton dimension 2

In [17]:
loss = nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)
print(output.backward())

None
